In [ ]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Optional: Print to verify (it should print part of the key or 'Found')
# DO NOT print the whole key in a shared notebook
if os.getenv("GOOGLE_API_KEY"):
    print("✅ API Key loaded successfully")
else:
    print("❌ API Key NOT found")

API Key loaded successfully


# 🏭 Meta Agent Factory System

## Overview
This notebook implements a **Meta Agent Factory** that uses multiple AI agents to design and build other AI agent systems automatically.

### Architecture:
1. **Consultant Agent** - Clarifies user requirements and proposes solutions
2. **Architect Agent** - Designs the system blueprint
3. **Builder Agent** - Generates executable Python code
4. **Storage Manager** - Persists blueprints and code
5. **Dynamic Executor** - Runs generated agents

---

## System Data Flow

```
┌─────────────────────────────────────────────────────────────────┐
│                         USER INPUT                              │
│              "I want a shopping agent"                           │
└────────────────────────┬────────────────────────────────────────┘
                         │
                         ▼
┌─────────────────────────────────────────────────────────────────┐
│                    CONSULTANT AGENT                             │
│  Proposes: Option 1, Option 2, Option 3                         │
│  Input:  User's vague request                                   │
│  Output: User's strategic choice + Refined requirements         │
└────────────────────────┬────────────────────────────────────────┘
                         │
                         ▼
┌─────────────────────────────────────────────────────────────────┐
│                    ARCHITECT AGENT                              │
│  Design → Propose → Get Feedback → Refine → Approve            │
│  Input:  Refined requirements                                   │
│  Output: JSON Blueprint { system_name, agents, flow_type }      │
│  Storage: blueprint.json                                        │
└────────────────────────┬────────────────────────────────────────┘
                         │
                         ▼
┌─────────────────────────────────────────────────────────────────┐
│                    BUILDER AGENT                                │
│  Generate → Code → Validate → Save                              │
│  Input:  JSON Blueprint                                         │
│  Output: Python code with agents configured                     │
│  Storage: agent.py                                              │
└────────────────────────┬────────────────────────────────────────┘
                         │
                         ▼
┌─────────────────────────────────────────────────────────────────┐
│               DYNAMIC EXECUTOR (importlib)                      │
│  Load → Instantiate → Run with user query                       │
│  Input:  Generated agent.py file                                │
│  Output: Agent executes user query                              │
└─────────────────────────────────────────────────────────────────┘
```

---

## Step 0: Verify Environment & Load Secrets
Load API keys from `.env` file (do NOT hardcode secrets in notebooks)

In [50]:
# [Task 1] Setup & Configuration
import os
import sys
import logging

# Force reload of dotenv (clear any cached values)
import importlib
if 'dotenv' in sys.modules:
    importlib.reload(sys.modules['dotenv'])

from dotenv import load_dotenv, dotenv_values

# Reload .env file (ignore cached values)
load_dotenv(override=True)

# Also load raw values to ensure they're fresh
env_vars = dotenv_values()
if "GEMINI_MODEL" in env_vars:
    os.environ["GEMINI_MODEL"] = env_vars["GEMINI_MODEL"]

# 1. Create Storage Directory for our Factory
FACTORY_STORAGE_PATH = "agent_factory_storage"
os.makedirs(FACTORY_STORAGE_PATH, exist_ok=True)
print(f"✅ Created storage directory: {FACTORY_STORAGE_PATH}")

# 2. Configure API Key (expects GOOGLE_API_KEY in .env or env vars)
if "GOOGLE_API_KEY" not in os.environ:
    raise RuntimeError("GOOGLE_API_KEY not set. Add it to your .env or environment variables.")

# 3. Load Gemini Model from .env (with fallback to safe default)
GEMINI_MODEL = os.getenv("GEMINI_MODEL", "gemini-2.5-flash-lite")
if not GEMINI_MODEL:
    GEMINI_MODEL = "gemini-2.5-flash-lite"
print(f"✅ Using model: {GEMINI_MODEL}")

# 4. Configure Logging (Helps us debug the factory itself)
logging.basicConfig(level=logging.INFO)
print("✅ Environment Configured")


from google.genai import types
from google.adk.models.google_llm import Gemini

# Define the retry configuration
retry_config = types.HttpRetryOptions(
    attempts=3, 
    exp_base=2, 
    initial_delay=1, 
    http_status_codes=[429, 500, 503]
)

# Helper function to create Gemini model (no need to repeat model name everywhere)
def get_gemini_model():
    """Returns a configured Gemini model instance."""
    return Gemini(model=GEMINI_MODEL, retry_options=retry_config)

print("✅ retry_config defined")
print("✅ get_gemini_model() helper ready")

✅ Created storage directory: agent_factory_storage
✅ Using model: gemma-3-27b-it
✅ Environment Configured
✅ retry_config defined
✅ get_gemini_model() helper ready


## Step 1: Core Setup & Configuration

**What happens here:**
- Create storage directory for agent blueprints and code
- Load API keys from environment variables
- Initialize retry configuration for API calls
- Create `get_gemini_model()` helper function (used by all agents)

**Why it matters:**
- Centralized model configuration: change model in `.env`, everything auto-updates
- Secure credential handling with dotenv
- Retry logic ensures robustness against API failures

In [51]:
# [Task 2] Storage Manager Class
import json
import os

class StorageManager:
    def __init__(self, base_path="agent_factory_storage"):
        self.base_path = base_path
        os.makedirs(base_path, exist_ok=True)
    
    def _get_agent_path(self, agent_name):
        return os.path.join(self.base_path, agent_name.lower().replace(" ", "_"))

    def save_blueprint(self, agent_name, blueprint_data):
        """Saves the JSON design of the agent."""
        path = self._get_agent_path(agent_name)
        os.makedirs(path, exist_ok=True)
        
        with open(os.path.join(path, "blueprint.json"), "w") as f:
            json.dump(blueprint_data, f, indent=4)
        print(f"💾 Blueprint saved for '{agent_name}'")

    def load_blueprint(self, agent_name):
        """Loads a saved blueprint if it exists."""
        path = os.path.join(self._get_agent_path(agent_name), "blueprint.json")
        if os.path.exists(path):
            with open(path, "r") as f:
                return json.load(f)
        return None

    def save_code(self, agent_name, code_content):
        """Saves the generated Python code."""
        path = self._get_agent_path(agent_name)
        os.makedirs(path, exist_ok=True)
        
        # Save as agent.py
        with open(os.path.join(path, "agent.py"), "w") as f:
            f.write(code_content)
        print(f"💾 Code saved to {path}/agent.py")

    def list_agents(self):
        """Lists all agents currently in storage."""
        if not os.path.exists(self.base_path):
            return []
        return [d for d in os.listdir(self.base_path) if os.path.isdir(os.path.join(self.base_path, d))]

# Initialize the storage
storage = StorageManager()
print("✅ Storage Manager Initialized")

✅ Storage Manager Initialized


## Step 2: Storage Manager

**What it does:**
- Manages persistent storage of agent blueprints (JSON) and generated code (Python)
- Organizes agents by name in the `agent_factory_storage/` directory

**Directory structure:**
```
agent_factory_storage/
├── agent_name_1/
│   ├── blueprint.json  (system design)
│   └── agent.py        (generated code)
├── agent_name_2/
│   ├── blueprint.json
│   └── agent.py
```

In [52]:
# [Task 4] The Architect Agent & Verification Tool
from google.adk.agents import Agent
from google.adk.tools import FunctionTool, ToolContext

# 1. Define the Verification Tool (The "Pause" Button)
def get_human_feedback(blueprint: str, tool_context: ToolContext):
    """
    Presents the proposed architecture blueprint to the human user and waits for feedback.
    
    Args:
        blueprint: A formatted string describing the proposed agents and flow.
    """
    # This checks if we are resuming from a pause
    if not tool_context.tool_confirmation:
        # PAUSE EXECUTION here and wait for human input
        tool_context.request_confirmation(
            hint=f"📋 ARCHITECT'S PROPOSAL:\n{blueprint}\n\n👉 Type 'Approved' to proceed, or type your changes (e.g., 'Add a verification agent').",
        )
        return {"status": "paused_for_feedback"}
    
    # If we are here, the human has responded!
    user_response = tool_context.tool_confirmation.payload
    print(f"\n👤 User Feedback Received: {user_response}")
    return {"user_feedback": user_response}

# 2. Define the Architect Agent
architect_agent = Agent(
    name="Architect",
    model=get_gemini_model(),
    instruction="""
    You are an expert AI Systems Architect using the Google Agent Development Kit (ADK).
    
    **Your Goal:** Design a valid JSON blueprint for a multi-agent system based on the user's request.
    
    **Your Process (Loop):**
    1. **Analyze:** Read the user's request or latest feedback.
    2. **Design:** Draft a JSON object with this structure:
       {
         "system_name": "string (snake_case)",
         "description": "string",
         "flow_type": "Sequential" or "Parallel",
         "agents": [
           { "name": "AgentName", "role": "string", "tools": ["tool_name"] }
         ]
       }
    3. **Verify:** You MUST call the `get_human_feedback` tool with a readable summary of your design.
    4. **Iterate:** - If the tool returns "Approved", output the final JSON.
       - If the tool returns feedback (e.g., "Add a search agent"), UPDATE your design and call `get_human_feedback` again.
    
    **Constraint:** Do NOT output the final JSON until you receive explicit "Approved" feedback.
    """,
    tools=[FunctionTool(get_human_feedback)],
    output_key="final_blueprint"
)

print("✅ Architect Agent & Verification Tool Created")

✅ Architect Agent & Verification Tool Created


## Step 3: Architect Agent & Verification Tool

**Purpose:**
- The **Architect** designs multi-agent system blueprints based on requirements
- The **Verification Tool** (`get_human_feedback`) acts as a "pause button" for human feedback

**Workflow:**
1. Architect proposes a JSON blueprint
2. User reviews and provides feedback
3. If feedback is "Approved" → finalize blueprint
4. If feedback is specific changes → architect updates design and loops

**Blueprint structure:**
```json
{
  "system_name": "agent_name",
  "description": "what it does",
  "flow_type": "Sequential or Parallel",
  "agents": [
    { "name": "Agent1", "role": "description", "tools": ["tool_name"] }
  ]
}
```

In [53]:
# [Task 3] The Consultant Agent
consultant_agent = Agent(
    name="Consultant",
    model=get_gemini_model(),
    instruction="""
    You are an AI Solutions Consultant. Your goal is to clarify the user's intent 
    and propose 2-3 distinct "Agent Architectures" they could build using the Google ADK.

    **If the user asks for a generic agent (e.g., "Shopping Agent"):**
    - Propose 2-3 specific variations.
      Example:
      1. "The Deal Hunter": Just searches and tracks prices. (Agents: Searcher, Tracker)
      2. "The Full Concierge": Searches, manages a cart, and buys. (Agents: Discovery, Cart, Buyer)
    - Ask the user to choose one or refine their request.

    **If the user asks for an update (e.g., "Add a verification step"):**
    - Confirm the specific change they want to make to the existing design.
    - Pass this requirement clearly to the Architect.

    Your output should be a clear set of options or a confirmed requirement definition.
    """,
    output_key="refined_requirement"
)

print("✅ Consultant Agent Created")

✅ Consultant Agent Created


## Step 4: Consultant Agent

**Role:**
- Clarifies vague user requests
- Proposes 2-3 specific architectural approaches
- Ensures user chooses a concrete path before Architect designs

**Example:**
- User: "I want a shopping agent"
- Consultant proposes:
  1. **Deal Hunter** - just searches prices
  2. **Full Concierge** - search, cart, and checkout
  3. **Price Tracker** - monitors prices over time
- User selects one → sent to Architect

In [54]:
# [Task 5] The Builder Agent (Strict Imports & Syntax)
builder_agent = Agent(
    name="Builder",
    model=get_gemini_model(),
    instruction="""
    You are a Senior Python Developer specializing in the Google Agent Development Kit (ADK).
    
    **Input:** A validated JSON Blueprint for an ADK Agent System.
    **Task:** Write the complete, executable Python code to implement this system.
    
    **CRITICAL RULES (Follow Strictly or the code will crash):**
    
    1. **IMPORTS (COPY THIS EXACTLY):**
       You MUST start the file with EXACTLY these imports. Do NOT add others.
       ```python
       import os
       from google.genai import types
       from google.adk.agents import Agent, LlmAgent, SequentialAgent, ParallelAgent
       from google.adk.models.google_llm import Gemini
       from google.adk.tools import google_search
       ```
       ❌ **FORBIDDEN:** Do NOT import `google.generativeai`. Do NOT import `google.api_core`.
    
    2. **RETRY CONFIG:**
       ```python
       retry_config = types.HttpRetryOptions(
           attempts=3, exp_base=2, initial_delay=1, http_status_codes=[429, 500, 503]
       )
       ```
    
    3. **AGENT DEFINITIONS:**
       - Use `LlmAgent` or `Agent`.
       - **Arguments:** `name="..."`, `instruction="..."` (NOT role), `model=Gemini(model=GEMINI_MODEL, retry_options=retry_config)`, `tools=[...]`.
       - **Tools:** If the blueprint asks for custom tools (like 'cart_manager'), create a simple dummy class for them:
         ```python
         class CartManager:
             def add_to_cart(self, item: str): return f"Added {item}"
         cart_tool = CartManager()
         # Then use tools=[cart_tool.add_to_cart]
         ```
    
    4. **ROOT AGENT:**
       - Define the pipeline agent (SequentialAgent or ParallelAgent) and name the variable `root_agent`.
    
    **Output:** ONLY the raw Python code. No markdown code fences (```python ... ```).
    """,
    output_key="generated_code"
)

print("✅ Builder Agent Updated (Strict Imports Enforced)")

✅ Builder Agent Updated (Strict Imports Enforced)


## Step 5: Builder Agent

**Job:**
- Takes the approved JSON blueprint from Architect
- Generates **production-ready Python code** using Google ADK
- Enforces strict import rules and syntax

**Key rules Builder enforces:**
- ✅ Only specific imports allowed
- ✅ Uses `SequentialAgent` or `ParallelAgent` based on flow_type
- ✅ All sub-agents properly configured with model & tools
- ❌ No hardcoded API keys in generated code
- ❌ No import of deprecated packages

## Step 6: The Factory Orchestrator

**Purpose:**
- Coordinates all three agents (Consultant → Architect → Builder)
- Handles multi-turn conversations with human feedback
- Manages error handling and JSON validation
- Saves outputs at each stage

**Three-phase workflow:**

```
PHASE 1: CONSULTANT
  ├─ Takes user prompt
  ├─ Proposes 2-3 architectural options
  └─ Collects user's strategic choice

PHASE 2: ARCHITECT
  ├─ Receives refined requirements
  ├─ Generates JSON blueprint
  ├─ Gets human feedback (approve/refine loop)
  └─ Finalizes blueprint when approved

PHASE 3: BUILDER
  ├─ Receives approved blueprint
  ├─ Generates Python code
  ├─ Saves code to agent_factory_storage/
  └─ Returns agent name for execution
```

**Key features:**
- 🔄 Iterative feedback loop - Architect refines until approved
- 💾 Auto-saves at each phase
- ⚠️ Robust error handling for JSON parsing
- 🎯 Validates blueprint structure before passing to Builder

In [55]:
# [Task 6] The Factory Orchestrator (Fixed)
from google.adk.runners import InMemoryRunner
from google.genai import types
import json

async def run_agent_factory(user_initial_prompt: str, agent_name_slug: str):
    """
    Orchestrates the creation of a new agent system.
    """
    print(f"\n🏭 FACTORY STARTED: Building '{agent_name_slug}'")
    print(f"🎯 Goal: {user_initial_prompt}")

    # --- PHASE 1: THE CONSULTANT (Strategy) ---
    print("\n\n--- 👔 STEP 1: CONSULTANT (Defining Strategy) ---")
    consultant_runner = InMemoryRunner(agent=consultant_agent)
    
    # Get strategic options
    response = await consultant_runner.run_debug(user_initial_prompt)
    
    # Safely get text from consultant
    consultant_text = "No suggestions available."
    if response and response[-1].content and response[-1].content.parts:
        for part in response[-1].content.parts:
            if part.text:
                consultant_text = part.text
                break

    print("\n👉 [ACTION REQUIRED] Read the Consultant's options above.")
    user_strategy = input("👤 Enter your choice or refinement (e.g., 'Option 2' or 'Add a price checker'): ")
    
    # Combine history for the Architect
    architect_input = (
        f"User Original Request: {user_initial_prompt}\n"
        f"Consultant's Analysis: {consultant_text}\n"
        f"User Strategic Choice: {user_strategy}"
    )

    # --- PHASE 2: THE ARCHITECT (Design Loop) ---
    print("\n\n--- 📐 STEP 2: ARCHITECT (Designing Blueprint) ---")
    architect_runner = InMemoryRunner(agent=architect_agent)
    
    current_input = architect_input
    final_blueprint = None
    
    while True:
        # Run the architect
        events = await architect_runner.run_debug(current_input)
        
        # --- ROBUST RESPONSE PARSING ---
        # We need to find the latest blueprint, whether it's in text or a tool call
        latest_blueprint_proposal = None
        
        # Check the last few events for a tool call to 'get_human_feedback'
        for event in reversed(events):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    # Case A: Architect called the tool (Most likely)
                    if part.function_call and part.function_call.name == "get_human_feedback":
                        args = part.function_call.args
                        # Arguments can be a dict or structure depending on SDK version
                        if isinstance(args, dict) and "blueprint" in args:
                            latest_blueprint_proposal = args["blueprint"]
                        elif hasattr(args, "blueprint"): # Handle object access
                            latest_blueprint_proposal = args.blueprint
                        break
                    # Case B: Architect just outputted text (Fallback)
                    if part.text and "system_name" in part.text:
                        latest_blueprint_proposal = part.text
            if latest_blueprint_proposal:
                break
        
        # Show the proposal if we found one
        if latest_blueprint_proposal:
            print(f"\n📋 ARCHITECT'S PROPOSAL:\n{latest_blueprint_proposal}")
        else:
            print("\n⚠️ Architect thinking... (No blueprint found in this turn)")

        # --- INTERACTION ---
        print("\n🤔 SYSTEM: Review the proposal above.")
        feedback = input("👤 Enter 'Approved' to build, or describe changes: ")
        
        if feedback.lower() == "approved":
            # If we found a blueprint in the tool call, use it. 
            if latest_blueprint_proposal:
                final_blueprint = latest_blueprint_proposal
            else:
                # Force one last output if missing
                final_step = await architect_runner.run_debug("Approved. Output the final JSON blueprint strictly.")
                final_blueprint = final_step[-1].content.parts[0].text
            
            # Clean up markdown code fences
            final_blueprint = final_blueprint.replace("```json", "").replace("```", "").strip()
            break
        else:
            # Loop back
            current_input = f"User Feedback: {feedback}. Update the design."

    # Save Blueprint
    try:
        blueprint_json = json.loads(final_blueprint)
        storage.save_blueprint(agent_name_slug, blueprint_json)
    except Exception as e:
        print(f"❌ Error parsing Architect's JSON: {e}")
        print("Saving raw text as backup...")
        # Save simple wrapper to not lose work
        storage.save_blueprint(agent_name_slug, {"raw_blueprint": final_blueprint})
        
    # --- PHASE 3: THE BUILDER (Coding) ---
    print("\n\n--- 👷 STEP 3: BUILDER (Writing Code) ---")
    builder_runner = InMemoryRunner(agent=builder_agent)
    
    code_events = await builder_runner.run_debug(f"Here is the approved blueprint: {final_blueprint}")
    
    # Robust code extraction
    generated_code = ""
    for part in code_events[-1].content.parts:
        if part.text:
            generated_code += part.text
    
    # Clean up markdown
    generated_code = generated_code.replace("```python", "").replace("```", "").strip()
    
    # Save Code
    storage.save_code(agent_name_slug, generated_code)
    
    print(f"\n✅ SYSTEM BUILT! Saved to: agent_factory_storage/{agent_name_slug}/agent.py")
    return agent_name_slug

print("✅ Orchestrator Function Updated (Fix Applied)")

✅ Orchestrator Function Updated (Fix Applied)


🚀 Task 7: The Dynamic Runtime
Now that we've built the code file, how do we run it immediately without restarting the notebook? We use Python's importlib to dynamically load the new file.

In [56]:
# [Task 7] Dynamic Executor & UI Launcher
import importlib.util
import sys

def load_and_run_agent(agent_name_slug, user_query):
    """
    Dynamically loads the generated agent file and runs it.
    """
    file_path = f"agent_factory_storage/{agent_name_slug}/agent.py"
    
    print(f"\n🚀 LAUNCHING: {agent_name_slug}...")
    
    # 1. Dynamic Import
    spec = importlib.util.spec_from_file_location("generated_module", file_path)
    module = importlib.util.module_from_spec(spec)
    sys.modules["generated_module"] = module
    spec.loader.exec_module(module)
    
    # 2. Get the 'root_agent' from the file
    if not hasattr(module, "root_agent"):
        print("❌ Error: Generated code does not define 'root_agent'.")
        return
    
    agent_instance = module.root_agent
    
    # 3. Create Runner and Run
    runner = InMemoryRunner(agent=agent_instance)
    
    print(f"💬 Sending Query: '{user_query}'")
    # We use a sync wrapper or just await if in Jupyter
    return runner.run_debug(user_query)

print("✅ Dynamic Executor Ready")

✅ Dynamic Executor Ready


## Step 7: Dynamic Runtime Executor

**What it does:**
- Loads the generated `agent.py` file **dynamically** at runtime
- Uses Python's `importlib` to avoid restarting the notebook
- Executes the agent with a user query

**Why dynamic loading?**
- No need to manually copy-paste generated code
- Agents can be updated and re-run without kernel restart
- Enables iterative development

In [57]:
# [Task 8] Run the Factory!
import time

# ... inside your loop or before calling the factory ...
time.sleep(5)

# 1. Build the Agent
# We will build a 'personal_shopper'
agent_id = await run_agent_factory(
    user_initial_prompt="I want a shopping agent that finds products and buys them.", 
    agent_name_slug="personal_shopper"
)

# 2. Run the Agent we just built
# (Only runs if build was successful)
if agent_id:
    print("\n\n✨ TESTING THE NEW AGENT ✨")
    await load_and_run_agent(agent_id, "Find a nice mechanical keyboard under $100")

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemma-3-27b-it, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:google_genai.models:AFC is enabled with max remote calls: 10.



🏭 FACTORY STARTED: Building 'personal_shopper'
🎯 Goal: I want a shopping agent that finds products and buys them.


--- 👔 STEP 1: CONSULTANT (Defining Strategy) ---

 ### Created new session: debug_session_id

User > I want a shopping agent that finds products and buys them.


INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemma-3-27b-it:generateContent "HTTP/1.1 400 Bad Request"


ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Developer instruction is not enabled for models/gemma-3-27b-it', 'status': 'INVALID_ARGUMENT'}}

---

## 📚 Quick Reference & Troubleshooting

### How to use this notebook:

**For first-time users:**
1. Run cell 1 (environment check)
2. Run cell 2 (setup & config)
3. Run cells 3-15 in order to initialize agents
4. Run cell 17 (run the factory)
5. Follow prompts to build your agent

**To change the model:**
- Edit `.env` file: `GEMINI_MODEL=gemini-2.5-flash-lite`
- Re-run cell 2 (setup) to reload

**To inspect generated agents:**
```bash
# List all built agents
ls -la agent_factory_storage/

# View a blueprint
cat agent_factory_storage/personal_shopper/blueprint.json

# View generated code
cat agent_factory_storage/personal_shopper/agent.py
```

### Common Issues:

| Issue | Solution |
|-------|----------|
| `GOOGLE_API_KEY not set` | Add key to `.env`: `GOOGLE_API_KEY=your_key` |
| Model not found error | Check `.env` model name, re-run cell 2 |
| Blueprint not saved | Check JSON syntax errors in Architect output |
| Agent not executing | Ensure blueprint defines `root_agent` variable |

### Customization:

**To modify agent behavior:**
- Edit the `instruction` field in agent definitions (cells 7, 9, 11)
- Re-run the agent initialization cells
- Retry the factory

**To add new tools:**
- Define tool function with proper type hints
- Pass to agent via `tools=[...]` parameter
- Update Builder's instructions with tool usage guide

### Architecture Decision Log:

- **Why 3 agents?** Separation of concerns: Strategy → Design → Implementation
- **Why iterative feedback?** Ensures output matches user intent
- **Why dynamic loading?** Enables fast iteration without kernel restart
- **Why JSON blueprints?** Language-agnostic design, easy version control

## Step 8: Run the Factory!

**This is the end-to-end test:**

1. **Initialization phase** - Factory starts and prompts for user input
2. **Consultant phase** - Clarifies requirements (you choose an option)
3. **Architect phase** - Designs blueprint (you approve or suggest changes)
4. **Builder phase** - Generates Python code automatically
5. **Storage phase** - Saves blueprint + code to `agent_factory_storage/`
6. **Execution phase** - Dynamically loads and runs the new agent

**Expected flow:**
```
🏭 FACTORY STARTED
  → Read user prompt
  → Consultant suggests options
  → You choose option
  → Architect designs blueprint
  → You approve/refine (loop if needed)
  → Builder writes code
  → Agent saved and tested
```

**Output files created:**
- `agent_factory_storage/personal_shopper/blueprint.json` - System design
- `agent_factory_storage/personal_shopper/agent.py` - Executable code